In [1]:
import pandas as pd
from nltk.tokenize import word_tokenize
import pysentiment2 as ps
from nltk.corpus import stopwords
import string

# 整理資料（將不需要的columns Drop掉）

In [2]:
df = pd.read_excel('/Users/sinjhanghuang/Documents/碩士畢業論文/柏凱聯合論文/柏凱合作文章2011-2015（整合版）/lpcr2comp2_all_data.xlsx')
df

,FacilityID,PackageID,TargetCompany,LoanType,PrimaryPurpose,SecondaryPurpose,FacilityAmt,Currency,ExchangeRate,FacilityStartDate,...,TobinQ,lg_TobinQ,logmvasset,lg_logmvasset,COGStoSales,lg_COGStoSales,roa1,lg_roa1,lg_sale,lg2_sale
0,7715,5006,NaN,Revolver/Term Loan,Work. cap.,Corp. purposes,10000000.0,United States Dollars,1.0,01/11/1990,...,1.0808,1.3917,6.017718,6.292893,0.7846,0.7574,0.0562,0.0982,444.875,406.360
1,9756,6460,NaN,Revolver/Line >= 1 Yr.,Corp. purposes,NaN,30000000.0,United States Dollars,1.0,04/16/1991,...,1.0201,1.0808,5.999714,6.017718,0.7833,0.7846,0.0345,0.0562,466.542,444.875
2,11662,7970,NaN,Revolver/Term Loan,Corp. purposes,Debt Repay.,30000000.0,United States Dollars,1.0,10/15/1991,...,1.0201,1.0808,5.999714,6.017718,0.7833,0.7846,0.0345,0.0562,466.542,444.875
3,45532,34433,NaN,Revolver/Term Loan,Debt Repay.,NaN,30000000.0,United States Dollars,1.0,09/09/1996,...,1.5572,1.3410,6.714958,6.375309,0.7950,0.8002,0.0623,0.0520,504.990,451.395
4,45541,34441,NaN,Revolver/Term Loan,Debt Repay.,Corp. purposes,30000000.0,United States Dollars,1.0,10/07/1996,...,1.5572,1.3410,6.714958,6.375309,0.7950,0.8002,0.0623,0.0520,504.990,451.395
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60990,330733,234876,NaN,Term Loan A,Corp. purposes,NaN,975000000.0,United States Dollars,1.0,10/15/2014,...,3.6273,3.1714,8.897298,8.744985,0.5596,0.5761,0.1329,0.1104,2093.500,2046.600
60991,331142,234876,NaN,Revolver/Line >= 1 Yr.,Corp. purposes,NaN,100000000.0,United States Dollars,1.0,10/15/2014,...,3.6273,3.1714,8.897298,8.744985,0.5596,0.5761,0.1329,0.1104,2093.500,2046.600
60992,347655,246460,NaN,Revolver/Line >= 1 Yr.,Corp. purposes,NaN,400000000.0,United States Dollars,1.0,09/30/2015,...,3.7559,3.6273,9.057577,8.897298,0.5478,0.5596,0.0916,0.1329,2118.300,2093.500
60993,347656,246460,NaN,Term Loan A,Corp. purposes,NaN,938437000.0,United States Dollars,1.0,09/30/2015,...,3.7559,3.6273,9.057577,8.897298,0.5478,0.5596,0.0916,0.1329,2118.300,2093.500


In [3]:
# 篩選2011-2015資料
df['year'] = df['FacilityStartDate'].apply(lambda x : x[-4:])
print(df[['year']])

       year
0      1990
1      1991
2      1991
3      1996
4      1996
...     ...
60990  2014
60991  2014
60992  2015
60993  2015
60994  2015

[60995 rows x 1 columns]


In [4]:
df = df.iloc[:,[0,9,31,40]]
df = df.loc[(df['year'] == "2011") | (df['year'] == "2012") | (df['year'] == "2013") | (df['year'] == "2014") | (df['year'] == "2015")]
df = df.drop(columns='year')
df = df.reset_index(drop=True)
df

,FacilityID,FacilityStartDate,company
0,270792,04/12/2011,AAR Corp
1,340002,03/24/2015,AAR Corp
2,302944,06/21/2013,American Airlines Inc
3,307808,08/01/2013,American Airlines Inc
4,313900,12/17/2013,American Airlines Inc
...,...,...,...
8560,330733,10/15/2014,Allegion Plc
8561,331142,10/15/2014,Allegion Plc
8562,347655,09/30/2015,Allegion Plc
8563,347656,09/30/2015,Allegion Plc


# 將公司名稱的縮寫去除

In [5]:
df['company_clear'] = None
for j in range(0,len(df['company'])):
    text = df.loc[j, 'company']
    exclude = ['Inc','Corp','Co','Ltd','LLC','LP','SA','Plc','Group','Inc.','&'] # 將縮寫建為一個list
    tokens = word_tokenize(text)                                             # 將公司名做tokens
    text = [ch for ch in tokens if ch not in exclude]                        # 如果公司名沒有所寫就加進text
    text = ' '.join(text)                                                    # 合併tokens
    df.loc[j, 'company_clear'] = text

# 將公司名稱裡的[ ]去除

In [6]:
# # 測試用正則表達式，去除公司名稱裡的 "(...)"&"[...]"
# import re
# org_string = df.iloc[279,3]
# pattern = r'^\W+$'
# mod_string = re.sub(pattern, '', org_string )# sub用法：將pattern所指定的字母取代
# print(mod_string)

In [7]:
# # 測試用split方式，去除公司名稱裡的 "(...)"&"[...]"
# org_string = df.iloc[278,3]
# org_string = org_string.split(" [")[0]
# org_string = org_string.split(" (")[0]
# org_string

In [8]:
for i in range(0,len(df)):
    org_string = df.loc[i, 'company_clear']
    org_string = org_string.split(" [")[0]
    org_string = org_string.split(" (")[0]
    df.loc[i, 'company_clear'] = org_string

In [9]:
df.to_excel('2011-2015 company_name_clear.xlsx', index=False)